In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [3]:
pre = pd.read_excel('EP47_45degree_FFP.xlsx')
next = pd.read_excel('EP47_45degree_PR.xlsx',sheet_name='Test Result')

In [4]:
pre.sample(3)

,ItemName Type,Process Type,Item,ItemName,Count/Count ESN,Retest%
141,17798,Epix_Pro_AutoATE,28,蜂鳴器V28|Beeper V28P0,3/568,0.005282
73,17800,Epix_Pro_CT,8,電池電壓|Battery Voltage,2/450,0.440000
152,17798,Epix_Pro_AutoATE,145,RGN245 GPS ( L1_L5),1/568,0.001761


In [5]:
next.sample(5)

,ItemName Type,Process Type,Item,ItemName,Count/Count ESN,Retest%
161,8837,Airtight_table,123,變形量絕對值超出規格_正壓,1/644,0.16
215,17798,Epix_Pro_AutoATE,35,Keypad Down,4/738,0.54
1,17819,Epix_Pro_FlashLight,14,Fixture ID lux White LED 80% before Calibratio...,128/549,23.32
129,17819,Epix_Pro_FlashLight,9,Set White Scaling Factor 100%,2/549,0.36
255,17858,Epix_Pro_ECG_ATE,16,WF_geneator Connection (2),1/767,0.13


# Tweak and Filter

Blacklist

In [6]:
nextocessTypeBlackList = 'Pack|Click|RabbitCard|EasyCard|DeleteBundle|nextodScan|FileCopyer'
ItemNameBlackList = 'ESN|Check nextoductionMap|Fixture ID'

tweak_filter def

In [17]:
def tweak_filter(raw):
    return(raw
    .drop('Comment',axis=1, errors='ignore')
    .rename(columns = lambda name : name.replace(" ",'').replace('/','').replace('%',''))
    .sort_values('Retest', ascending=False)
    .astype({'ItemNameType':'category','ProcessType':'category','Item':'category','ItemName':'category'})
    .assign(CountESN = lambda df : df.CountCountESN.str.split('/').str[1].astype('int16'))
    .query("~ProcessType.str.contains(@nextocessTypeBlackList) and ~ItemName.str.contains(@ItemNameBlackList)")
    # .CountESN.quantile(0.03)
    .query("CountESN > CountESN.quantile(0.05)")
    .query("Retest >= Retest.mean()")
    # .Retest.mean()
    )

filter def

In [8]:
def tweak(raw):
    return(raw
    .drop('Comment',axis=1, errors='ignore')
    .rename(columns = lambda name : name.replace(" ",'').replace('/','').replace('%',''))
    .sort_values('Retest', ascending=False)
    .astype({'ItemNameType':'category','ProcessType':'category','Item':'category','ItemName':'category'})
    .assign(CountESN = lambda df : df.CountCountESN.str.split('/').str[1].astype('int16'))
    )

In [21]:
# tweak_filter(next).to_excel('tweak_EP47_45degree_PR.xlsx', index=False)

# Merge two phases

In [9]:
pre_next = pd.merge(
    left=tweak_filter(pre), right=tweak(next), how='left',
    left_on=['ItemNameType','ProcessType','Item','ItemName'], right_on=['ItemNameType','ProcessType','Item','ItemName'],
    suffixes=['_pre','_next'])

In [10]:
pre_next.to_excel('output.xlsx', index=False)

# RetryImprove def

In [11]:
def RetryImprove(df):
    return((100 * (df.Retest_pre.sum() - df.Retest_next.sum()) / df.Retest_pre.sum()).round(3))

In [12]:
RetryImprove(pre_next)

-40.191

In [13]:
def RetryDiff(df):
    return(df
     .assign(diff = df.Retest_pre - df.Retest_next)
     [['ProcessType','ItemName','diff']]
     .sort_values('diff', ascending=False)
     .dropna(axis=0)
     .style.bar('diff', color=['red','green'])
     )
RetryDiff(pre_next)

,ProcessType,ItemName,diff
2,Epix_Pro_FT2,ANT_power 2450MHz BY REF (sapphire),5.940000
16,Epix_Pro_FT1,006J Comparison,0.880000
21,Epix_Pro_Baro,1 ATM diff with golden,0.760000
15,Epix_Pro_FT2,BLE LESC test|藍芽LESC測試,0.460000
20,Epix_pro_RTESN,光學心跳模組檢查|Check OHRM LED,0.210000
13,Epix_Pro_ECG_ATE,Motor Resonant Frequency,-0.020000
7,Epix_Pro_FT2,Manifest Event Test,-1.950000
0,Epix_Pro_ECG_FT,ECG impedance short,-2.270000
19,Epix_Pro_ATE_TI,AFE Power Consumption data,-6.330000
10,Epix_Pro_FT1,電池電壓|Battery Voltage,-7.230000
